In [1]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# %cd "/content/drive/MyDrive"

In [2]:
import datetime
import pandas as pd
import math
import tensorflow as tf
import numpy as np
import torch
import random
import matplotlib.pyplot as plt
import os

from tensorflow.python.client import device_lib
from torch.utils.data import DataLoader, Dataset, Subset, ConcatDataset

In [3]:
!nvidia-smi

Mon Dec 25 11:13:35 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090        Off | 00000000:65:00.0 Off |                  N/A |
| 60%   52C    P2             133W / 370W |  22684MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
gpus = tf.config.list_physical_devices(device_type='GPU')
tf.config.set_visible_devices(devices=gpus[0], device_type='GPU')

In [5]:
print(device_lib.list_local_devices())
print("===============")
print(torch.__version__)
print("===============")
print(torch.version.cuda)
print("===============")
print(torch.backends.cudnn.version())
print("===============")
print(tf.__version__)
print("===============")
print(tf.config.list_physical_devices('GPU'))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18295020351171411350
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1142882304
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2010853718315646423
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6"
xla_global_id: 416903419
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 23283367936
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13759530565716513981
physical_device_desc: "device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:b3:00.0, compute capability: 8.6"
xla_global_id: 2144165316
]
2.1.2+cu121
12.1
8902
2.8.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


2023-12-25 11:13:35.205345: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-25 11:13:35.348277: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:0 with 1089 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6
2023-12-25 11:13:35.348856: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /device:GPU:1 with 22204 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:b3:00.0, compute capability: 8.6


In [6]:
# Fix the seed for reproducibility
seed = 999
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

# data preprocess

In [7]:
# filename = 'csv.tfrecords'

# raw_dataset = tf.data.TFRecordDataset(filename)
# X_train = []
# Y_train = []
# i=0
# for raw_record in raw_dataset:
#     sub_y = []
#     example = tf.train.Example()
#     example.ParseFromString(raw_record.numpy())
#     print(example)
#     # print(example.features.feature['icu_intime'].bytes_list)
#     # sub_y.append(0 if example.features.feature['gender'].float_list.value[0] == 0 else 1)
#     break

In [8]:
df  = pd.read_csv("final_result.csv")
print(df.head(5))

   Unnamed: 0  index   stay_id   hadm_id  subject_id  label  \
0           0      0  30039798  27880403    18499026      0   
1           1      0  30039798  27880403    18499026      0   
2           2      0  30039798  27880403    18499026      0   
3           3      0  30039798  27880403    18499026      0   
4           4      0  30039798  27880403    18499026      0   

            icu_intime  charttime  urineoutput          icu_outtime  \
0  2137-09-15 08:36:00          0          0.0  2137-09-16 17:28:21   
1  2137-09-15 08:36:00          1        185.0  2137-09-16 17:28:21   
2  2137-09-15 08:36:00          2         82.0  2137-09-16 17:28:21   
3  2137-09-15 08:36:00          3        220.0  2137-09-16 17:28:21   
4  2137-09-15 08:36:00          4        140.0  2137-09-16 17:28:21   

   aki_stage  aki_charttime  rank  
0        0.0              0     1  
1        0.0              1     1  
2        0.0              2     1  
3        0.0              3     1  
4        0.0  

In [9]:
# with tf.device('/cpu:0'):

#     filename = 'csv.tfrecords'
    
#     raw_dataset = tf.data.TFRecordDataset(filename)
#     # uo = data['urineoutput']
#     # aki_stage = data['aki_stage']
#     # charttime = data['charttime']
#     uo = []
#     aki_stage = []
#     charttime = []
#     stay_id_list = set()
#     i=0
#     for raw_record in raw_dataset:
#         sub_y = []
    
#         example = tf.train.Example()
#         example.ParseFromString(raw_record.numpy())
#         stay_id_list.add(example.features.feature['stay_id'].float_list.value[0])
#         # uo.append(example.features.feature['urineoutput'].float_list.value[0])
#         # aki_stage.append(example.features.feature['aki_stage'].float_list.value[0])
#         # charttime.append(example.features.feature['charttime'].float_list.value[0])
        
#     print(len(stay_id_list))
#     # print(len(uo))
#     # print(len(stay_id_list))
#     # X_train = np.array(X_train)
#     # Y_train = np.array(Y_train)
#     # X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],X_train.shape[2],-1)
#     # Y_train = Y_train.astype(np.float64)
#     # print(Y_train.shape)



### padding data of each patient into the same length

In [10]:
stay_id_list = df["stay_id"].unique()
print("there are totally {} patients".format(len(stay_id_list)))

there are totally 49669 patients


In [11]:
# # Parameters for processing
# padding_hr = 23
# predict_after_hr = 6
# X_per_patient = []
# Y_per_patient = []

# # Process data for each stay_id
# for stay_id, data in stay_id_data.items():
#     uo = data['urineoutput']
#     aki_stage = data['aki_stage']
#     charttime = data['charttime']

#     if len(aki_stage) <= padding_hr + predict_after_hr:
#         continue

#     for i in range(len(aki_stage)):
#         if i + padding_hr > len(aki_stage) - 1:
#             break
#         aki_tmp = aki_stage[padding_hr + predict_after_hr]

#         uo_tmp = np.array(uo[i: i + padding_hr + 1])
#         time_tmp = np.array(charttime[i: i + padding_hr + 1])

#         uo_tmp = uo_tmp[:, np.newaxis]
#         time_tmp = time_tmp[:, np.newaxis]
#         x_tmp = np.hstack((uo_tmp, time_tmp))

#         X_per_patient.append(x_tmp)
#         Y_per_patient.append(aki_tmp)

# # Convert to NumPy arrays
# X_per_patient = np.array(X_per_patient, dtype=object)
# Y_per_patient = np.array(Y_per_patient, dtype=object)

# print("After processing, there are {} patients' data".format(len(X_per_patient)))
# print("X_per_patient shape:", X_per_patient.shape)
# print("Y_per_patient shape:", Y_per_patient.shape)

In [12]:
# padding_hr = 23          # the first `padding_hr` nodes have no output
# predict_after_hr = 6    # for each node, input the nth Hour urineOutput, output the (n + predict_after_hr)th Hour AKI_stage

# X_per_patient = [] # urine output from 0H ~ (N - predict_after_hr)H
# Y_per_patient = [] # AKI stage from (padding_hr + predict_after_hr)H ~ (N)H
# for stay_id in stay_id_list:
#     # for each patient
#     now_process = df.loc[df['stay_id'] == stay_id].reset_index()

#     uo = now_process["urineoutput"].tolist()
#     aki_stage = now_process["aki_stage"].tolist()
#     charttime = now_process["charttime"].tolist()

#     if len(aki_stage) <=  padding_hr + predict_after_hr: # there is no enough data for predict
#         continue


#     ## sliding window, split each 24 hour as a data
#     for i in range(len(aki_stage)):
#         if i + padding_hr > len(aki_stage) - 1:
#             break
#         # aki_tmp = aki_stage[padding_hr + predict_after_hr]
#         # if len(uo) - len(aki_stage) != padding_hr:
#         #     print("err")

#         aki_stage = np.array(aki_stage)
#         aki_stage[aki_stage == 2] = 1
#         aki_stage[aki_stage == 3] = 1
#         aki_stage[aki_stage == 4] = 2
        
#         uo_tmp = np.array(uo[i : i + padding_hr + 1])
#         time_tmp = np.array(charttime[i : i + padding_hr + 1])

#         uo_tmp = uo_tmp[:, np.newaxis]
#         time_tmp = time_tmp[:, np.newaxis]
#         x_tmp = np.hstack((uo_tmp, time_tmp))

#         X_per_patient.append(x_tmp)
#         # Y_per_patient.append(aki_tmp)
#         Y_per_patient.append(aki_stage)

# print("after process, there are {} patients' data".format(len(X_per_patient)))


# X_per_patient = np.array(X_per_patient)
# Y_per_patient = np.array(Y_per_patient)

# # the (dim of X) should be the (dim of Y) - padding_hr
# print(X_per_patient.shape)
# print(Y_per_patient.shape)


In [13]:
import numpy as np
padding_hr = 23          # the first `padding_hr` nodes have no output
predict_after_hr = 6  
X_per_patient = []  # urine output from 0H ~ (N - predict_after_hr)H
Y_per_patient = []  # AKI stage from (padding_hr + predict_after_hr)H ~ (N)H

for stay_id in stay_id_list:
    # for each patient
    now_process = df.loc[df['stay_id'] == stay_id].reset_index()

    uo = now_process["urineoutput"].tolist()
    aki_stage = np.array(now_process["aki_stage"].tolist())
    charttime = now_process["charttime"].tolist()

    if len(aki_stage) <= padding_hr + predict_after_hr:  # there is not enough data for prediction
        continue

    # Adjust AKI stages: Set 2 and 3 to 1, assuming other values are already 0 or 1
    aki_stage[(aki_stage == 2) | (aki_stage == 3)] = 1

    for i in range(len(aki_stage)):
        if i + padding_hr + predict_after_hr > len(aki_stage) - 1:
            break

        # Selecting the appropriate AKI stage value for this window
        aki_tmp = aki_stage[i + padding_hr + predict_after_hr]

        uo_tmp = np.array(uo[i : i + padding_hr + 1])
        time_tmp = np.array(charttime[i : i + padding_hr + 1])

        uo_tmp = uo_tmp[:, np.newaxis]
        time_tmp = time_tmp[:, np.newaxis]
        x_tmp = np.hstack((uo_tmp, time_tmp))

        X_per_patient.append(x_tmp)
        Y_per_patient.append(aki_tmp)


print("After processing, there are {} patients' data".format(len(X_per_patient)))

X_per_patient = np.array(X_per_patient, dtype=object)  # Use dtype=object for varying lengths
Y_per_patient = np.array(Y_per_patient)

print(X_per_patient.shape)
print(Y_per_patient.shape)


After processing, there are 2499542 patients' data
(2499542, 24, 2)
(2499542,)


In [23]:
print(len(Y_per_patient))
print(Y_per_patient)

2499542
[[[0.0]]

 [[0.0]]

 [[0.0]]

 ...

 [[0.0]]

 [[0.0]]

 [[1.0]]]


In [15]:
Y_per_patient = Y_per_patient[:, np.newaxis, np.newaxis]
print(Y_per_patient.shape)

(2499542, 1, 1)


In [16]:
AKI_patient = 0
noAKI_patient = 0
for AKI_stages in Y_per_patient:
    if 1 in AKI_stages or 2 in AKI_stages or 3 in AKI_stages:
        AKI_patient += 1
    else:
        noAKI_patient += 1

print("therea are {} windows with AKI".format(AKI_patient))
print("therea are {} windows without AKI".format(noAKI_patient))

therea are 619198 windows with AKI
therea are 1880344 windows without AKI


### Split Data

In [17]:
def shuffle(X,Y):
    randomList = np.arange(X.shape[0])
    np.random.shuffle(randomList)
    return X[randomList], Y[randomList]

X_per_patient, Y_per_patient = shuffle(X_per_patient, Y_per_patient)

In [18]:
def splitData(X, Y, val_rate, test_rate):
    X_val = X[ : int(X.shape[0]*val_rate)]
    Y_val = Y[ : int(Y.shape[0]*val_rate)]

    X_test = X[int(X.shape[0]*val_rate) : int(X.shape[0]*val_rate)+int(X.shape[0]*test_rate)]
    Y_test = Y[int(Y.shape[0]*val_rate) : int(Y.shape[0]*val_rate)+int(Y.shape[0]*test_rate)]

    X_train = X[int(X.shape[0]*val_rate)+int(X.shape[0]*test_rate) : ]
    Y_train = Y[int(Y.shape[0]*val_rate)+int(Y.shape[0]*test_rate) : ]
    return X_train, Y_train, X_val, Y_val, X_test, Y_test

In [19]:
X_train, Y_train, X_val, Y_val, X_test, Y_test = splitData(X_per_patient, Y_per_patient, val_rate=0.2, test_rate=0.1)
print(X_train.shape)
print(Y_train.shape)
print(X_val.shape)
print(Y_val.shape)
print(X_test.shape)
print(Y_test.shape)

(1749680, 24, 2)
(1749680, 1, 1)
(499908, 24, 2)
(499908, 1, 1)
(249954, 24, 2)
(249954, 1, 1)


# build LSTM model

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Lambda
from tensorflow.keras.optimizers import Adam
from torch import nn
from torch.nn import functional as F
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import backend


In [21]:
from keras.layers import Dropout
from keras.layers import Dense
import keras
def LSTMmodel():
    model = Sequential()
    model.add(LSTM(units=150, return_sequences = True, input_shape=((padding_hr + 1), 2)))
    model.add(Lambda(lambda x: x[:, -1:, :])) # use Lambda() to  remove output of the first `padding_hr` nodes
    #adding layer
    model.add(LSTM(units=150, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=100, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=100, return_sequences = True))
    model.add(Dropout(0.2))
    model.add(LSTM(units=100, return_sequences = True))
    model.add(Dropout(0.2))
    # model.add(Dense(units = 1))

    model.add(TimeDistributed((Dense(2, activation='softmax'))))

    # lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    #                 initial_learning_rate=1e-4,
    #                 decay_steps=14677,
    #                 decay_rate=0.9)

    # Compiling
    # regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')
    opt = Adam(learning_rate=0.001, clipvalue=1.0)
    model.compile(optimizer=opt,\
                  loss='sparse_categorical_crossentropy',\
                  # loss='binary_crossentropy',\ 
                  
                  # when keras.losses.BinaryCrossentropy(from_logits=True) with dense unit = 1, accuracy become 0.35
                  # change the last layer of LSTM units from 50 to 100 and change the loss function back to sparse_categorical, accu become 0.66 and 
                  # initial accu of the 24 hour with 5 layers is 0.71
                  
                  # loss = 'sparse_categorical_crossentropy',\
                  metrics=['accuracy'])
    return model

LSTMmodel = LSTMmodel()
LSTMmodel.summary()
LSTMmodel.save_weights("checkpoint.weights.0.h5")

2023-12-25 11:17:29.395521: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1089 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:65:00.0, compute capability: 8.6


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 24, 150)           91800     
                                                                 
 lambda (Lambda)             (None, 1, 150)            0         
                                                                 
 lstm_1 (LSTM)               (None, 1, 150)            180600    
                                                                 
 dropout (Dropout)           (None, 1, 150)            0         
                                                                 
 lstm_2 (LSTM)               (None, 1, 100)            100400    
                                                                 
 dropout_1 (Dropout)         (None, 1, 100)            0         
                                                                 
 lstm_3 (LSTM)               (None, 1, 100)            8

In [22]:
import tensorflow as tf
import gc
# This approach would typically be implemented with a ReduceLROnPlateau callback rather than a LearningRateScheduler
from tensorflow.keras.callbacks import ReduceLROnPlateau
gc.collect()
tf.config.run_functions_eagerly(True)

EScallback = EarlyStopping(monitor="val_loss", patience=10, verbose=1, mode="auto")
CKPTcallback = ModelCheckpoint(filepath="ckpt/checkpoint.weights.{epoch}.h5", monitor='val_loss', \
                               verbose=1, save_best_only=True, save_weights_only=True, mode="auto")



# Reduce learning rate when a metric has stopped improving
RLcallback = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

# Create a ReduceLROnPlateau callback
reduce_lr_on_plateau = ReduceLROnPlateau(
    monitor='val_loss',    # Monitor the validation loss
    factor=0.1,            # Factor by which the learning rate will be reduced. new_lr = lr * factor
    patience=5,            # Number of epochs with no improvement after which learning rate will be reduced
    min_lr=0.0001,         # Lower bound on the learning rate
    verbose=1              # Print messages when the callback takes an action
)

def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(-0.1)
LRcallback = LearningRateScheduler(scheduler)

# LSTMmodel.load_weights("ckpt/checkpoint.weights.0.h5")

history_callback = LSTMmodel.fit(X_train, Y_train, epochs=100, batch_size=128, validation_data=(X_val, Y_val), callbacks=[EScallback, CKPTcallback, reduce_lr_on_plateau])

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [ ]:
print(history_callback.history.keys())
total_epochs = len(history_callback.history["loss"])


fig, ax = plt.subplots(1, 1, figsize=(10, 10), dpi=100)
ax.set_title('Loss')
ax.plot(range(total_epochs), history_callback.history["loss"], label='Train')
ax.plot(range(total_epochs), history_callback.history["val_loss"], label='Valid')
ax.legend()
plt.show()
fig.savefig('metrics.jpg')
plt.close()

# try to evaluate by test set

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import warnings
warnings.filterwarnings("ignore")


In [ ]:
# LSTMmodel.load_weights("ckpt/checkpoint.{}.weights.h5".format(121))
LSTMmodel.load_weights("checkpoint.weights.0.h5")
y_pred = LSTMmodel.predict(X_test)

y_pred_oneClass = np.argmax(y_pred, axis=2)
Y_test_2D = Y_test.reshape((Y_test.shape[0], Y_test.shape[1]))

### all time accuracy (with -1 padding)

In [ ]:
### 5 classes
y_pred_flatten = y_pred_oneClass.reshape((-1))
y_test_flatten = Y_test_2D.reshape((-1))

print(classification_report(y_test_flatten, y_pred_flatten))
print(accuracy_score(y_test_flatten, y_pred_flatten))
print(confusion_matrix(y_test_flatten, y_pred_flatten))

print("=================")

### 3 classes
y_pred_flatten_3 = np.array(y_pred_flatten)
y_test_flatten_3 = np.array(y_test_flatten)

y_pred_flatten_3[y_pred_flatten_3 == 2] = 1
y_pred_flatten_3[y_pred_flatten_3 == 3] = 1
y_test_flatten_3[y_test_flatten_3 == 2] = 1
y_test_flatten_3[y_test_flatten_3 == 3] = 1

print(classification_report(y_test_flatten_3, y_pred_flatten_3))
print(accuracy_score(y_test_flatten_3, y_pred_flatten_3))
print(confusion_matrix(y_test_flatten_3, y_pred_flatten_3))


### without -1 padding

In [ ]:
### 5 classes
y_pred_nopad = []
y_test_nopad = []
for yp, yt in zip(y_pred_oneClass, Y_test_2D):
    yt_tmp = np.delete(yt, np.where(yt == 4))
    yp_tmp = yp[: yt_tmp.shape[0]]

    y_test_nopad = [*y_test_nopad, *yt_tmp]
    y_pred_nopad = [*y_pred_nopad, *yp_tmp]


print(classification_report(y_test_nopad, y_pred_nopad))
print(accuracy_score(y_test_nopad, y_pred_nopad))
print(confusion_matrix(y_test_nopad, y_pred_nopad))

print("=============")
### 3 classes
y_pred_nopad_3 = np.array(y_pred_nopad)
y_test_nopad_3 = np.array(y_test_nopad)

y_pred_nopad_3[y_pred_nopad_3 == 2] = 1
y_pred_nopad_3[y_pred_nopad_3 == 3] = 1
y_test_nopad_3[y_test_nopad_3 == 2] = 1
y_test_nopad_3[y_test_nopad_3 == 3] = 1

print(classification_report(y_test_nopad_3, y_pred_nopad_3))
print(accuracy_score(y_test_nopad_3, y_pred_nopad_3))
print(confusion_matrix(y_test_nopad_3, y_pred_nopad_3))

### without -1 padding, last 5 hours


In [ ]:
### 5 classes
y_pred_last5 = []
y_test_last5 = []
for yp, yt in zip(y_pred_oneClass, Y_test_2D):
    yt_tmp = np.delete(yt, np.where(yt == 4))
    yp_tmp = yp[: yt_tmp.shape[0]]

    y_test_last5 = [*y_test_last5, *yt_tmp[-5:]]
    y_pred_last5 = [*y_pred_last5, *yp_tmp[-5:]]


print(classification_report(y_test_last5, y_pred_last5))
print(accuracy_score(y_test_last5, y_pred_last5))
print(confusion_matrix(y_test_last5, y_pred_last5))

print("=============")
### 3 classes
y_pred_last5_3 = np.array(y_pred_last5)
y_test_last5_3 = np.array(y_test_last5)

y_pred_last5_3[y_pred_last5_3 == 2] = 1
y_pred_last5_3[y_pred_last5_3 == 3] = 1
y_test_last5_3[y_test_last5_3 == 2] = 1
y_test_last5_3[y_test_last5_3 == 3] = 1

print(classification_report(y_test_last5_3, y_pred_last5_3))
print(accuracy_score(y_test_last5_3, y_pred_last5_3))
print(confusion_matrix(y_test_last5_3, y_pred_last5_3))